In [16]:
#MAKE Simulated sequence networks for evolution experiments to display sequence variation in cytoscape
import os
os.putenv('DISPLAY', ':0.0')
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pylab as ply
import seaborn as sns
import cProfile
import random
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
%matplotlib inline
import CommonMotifProb

In [46]:
NTevolveprobDICT1={'A':[1-(1.8*10**-5+1.5*10**-5+2.0*10**-4), 1.8*10**-5, 1.5*10**-5, 2.0*10**-4],
                  'T':[1.4*10**-5, 1-(2.3*10**-4+3.5*10**-5+1.4*10**-5), 2.3*10**-4, 3.5*10**-5],
                  'C':[7.7*10**-6, 2.7*10**-5, 1-(7.7*10**-6+2.7*10**-5+5.1*10**-6), 5.1*10**-6],
                  'G':[3.1*10**-5, 3.5*10**-5, 5.4*10**-5, 1-(3.1*10**-5+3.5*10**-5+5.4*10**-5)]}
NTevolveprobDICT=NTevolveprobDICT1
PDICT={}
nts=['A','T','C','G']
for i in nts:
    for j in range(len(nts)):
        PDICT[i+nts[j]]=NTevolveprobDICT[i][j]
probDICT={}
for i in CommonMotifProb.DNACodontoAADICT:
    for j in CommonMotifProb.DNACodontoAADICT:
        if '-' not in i and '-' not in j:

            probDICT[i+j]=PDICT[i[0]+j[0]]*PDICT[i[1]+j[1]]*PDICT[i[2]+j[2]]

In [17]:
#for each codon at a given motif position eg S/T: compare it with all other codons: If the difference is one, calculate
#the probability either given the normal metric or the experimental mutation rate and save codon1 -> interaction -> codon2

def makeCodonnetwork(codonlist):
    interactList=[['Codon 1','Interaction', 'Codon2']]
    for codon1 in codonlist:
        for codon2 in codonlist:
            if CommonMotifProb.SeqMatch(codon1,codon2)==1:
                interactWeight=probDICT[codon1+codon2]
                interactList.append([codon1,interactWeight,codon2])
    return interactList

In [57]:
#for each codon at a given motif position eg S/T: compare it with all other codons: If the difference is one, calculate
#the probability either given the normal metric or the experimental mutation rate and save codon1 -> interaction -> codon2

def makeSeqnetwork(seqlist):
    interactList=[]
    uniquesource=[]
    for seq1 in seqlist:
        
        uniquesource.append(seq1)
        for seq2 in seqlist:
            if CommonMotifProb.SeqMatch(seq1,seq2)==1:
                interactWeight=1
                for l in range(len(seq1)):
                    interactWeight*=PDICT[seq1[l]+seq2[l]]
                interactList.append([seq1,interactWeight,seq2])
    return interactList

In [18]:
def ismotifmatch(seq,motif):
    for i in range(len(seq)):
        if seq[i] not in motif[i]:
            return False
    return True

In [19]:
def createmotif(motif):
    finalmotif=[]
    for i in motif:
        if i[0]=='.':
            finalmotif.append(list(CommonMotifProb.AminoAcid1LTR))
        elif i[0]=='^':
            print i
            aas=i[1:]
            finalmotif.append(list(CommonMotifProb.AminoAcid1LTR-set(aas)))
        else:
            finalmotif.append(i)
    return finalmotif

In [ ]:
import csv
GetCodonDict=CommonMotifProb.AAtoCodonDICT
motifAAs=['SER', 'THR']

codonlist=[item for sublist in motifAAs for item in GetCodonDict[sublist]]

CodonNet=makeCodonnetwork(codonlist)
print CodonNet
with open('CodonInteractSERTHR.tsv', "w") as output_handle:
    tsv_writer = csv.writer(output_handle, delimiter='\t')
    for i in CodonNet:  
        tsv_writer.writerow(i)

In [1]:
import pandas as pd
#NeutralSamplingDiffRatesNoBias_forMS1-0.csv
DFlist=[pd.read_csv('NeutralSamplingDiffGenotypesNew_forMS1-'+str(i)+'.csv',index_col=0) for i in range(10)]#,pd.read_csv('PKAmotifsimAltRate500reps_2.csv',index_col=0)]
#DFlist2=[pd.read_csv('NeutralSamplingDiff2AAs_forMS1-'+str(i)+'.csv',index_col=0) for i in range(10)]
df=pd.concat(DFlist)#+DFlist2)
#df= pd.read_csv('NeutralSamplingDiffGenotypes_forMS1-0.csv',index_col=0)

In [95]:

df2=df[(df['StartCodon']=='AGGAGGATATTGAGG')]#|(df['StartCodon']=='AGGAGGATATTGAGG')]
a=df2['TargetCodon'].value_counts()
print a

AGGAGGATATTGAGG    9960
AGGAGGATACTGAGG    9238
AGGAGGACATTGAGG    9212
AGGAGGATATCGAGG    9182
AGGAGGATATTGGGG    8923
GGGAGGATATTGAGG    8918
AGGAGGGTATTGAGG    8887
AGGAGGATGTTGAGG    8872
AGGGGGATATTGAGG    8864
AGGACGATATTGAGG    4655
AGGAGGATATTCAGG    4604
ACGAGGATATTGAGG    4578
AGGAGGATATTGAGC    4539
AGGAGCATATTGAGG    4535
AGCAGGATATTGAGG    4508
AGGAGGATATTGACG    4490
ATGAGGATATTGAGG    3343
AGGAGGATATTGAGT    3331
AGGAGTATATTGAGG    3287
AGGAGGATATTTAGG    3286
AGGATGATATTGAGG    3280
AGTAGGATATTGAGG    3253
AGGAGGATAGTGAGG    3245
AGGAGGATATGGAGG    3230
AGGAGGATATTGATG    3221
AGGAGGAGATTGAGG    3186
AGGAAGATATTGAGG    3000
AGGAGGATATTGAAG    2981
AGGAGGATATTGAGA    2952
AGAAGGATATTGAGG    2930
                   ... 
AGCCGGATATTGAGA       1
AAAAGGATGTTGAGG       1
AGGAGGATTGCGAGG       1
AGGAGGATAGTGTGG       1
AGGACGATATTGCGG       1
AGGAGAATAGTGAGG       1
GGGAGGACATTGGGG       1
AAGAGGCAATTGAGG       1
TTGAGGATATTGAGG       1
AGAAGGCTATTGAGG       1
AGGAGGATTTTGAGT 

In [30]:
df

,StartCodon,PopReplicate,TargetCodon,Count
0,CGACGAATCCTCCGA,1,CGACGAATCCTCCGA,11027
1,CGACGAATCCTCCGA,1,CGTCGAATCCTCCGA,30
2,CGACGAATCCTCCGA,1,CGACGAGTCCTCCGA,2
3,CGACGAATCCTCCGA,1,CGACGAATCCGCCGA,1
4,CGACGAATCCTCCGA,1,CGACGAATCCTCCGG,1
5,CGACGAATCCTCCGA,1,CGACGAATCCTCCTA,2
6,CGACGAATCCTCCGA,1,CGGCGAATCCTCCGA,2
7,CGACGAATCCTCCGA,1,CGACGGATCCTCCGA,4
8,CGACGAATCCTCCGA,1,CGATGAATCCTCCGA,1
9,CGACGAATCCTCCGA,1,CGACGAATCATCCGA,1


In [31]:
from CommonMotifProb import SeqMatch
seqnetworklist=[]
uniquesource=[]
for i in a.keys():
    uniquesource.append(i)
    for j in a.keys():
        if SeqMatch(i,j)==1 and j not in uniquesource:
            seqnetworklist.append([i,j])

In [96]:
seqnetworklist=makeSeqnetwork(a.keys())
columns=['SourceSeq','ProbWeight', 'TargetSeq']
#seqnetworklist

df2=pd.DataFrame(seqnetworklist,columns=columns)

In [89]:
dfnewaa

,SourceSeq,ProbWeight,TargetSeq
0,RRILR,0.004362,RRTLR
1,RRILR,0.004132,RRIPR
2,RRILR,0.002994,RRVLR
3,RRILR,0.000593,RRILP
4,RRILR,0.000647,RPILR
5,RRILR,0.000916,PRILR
6,RRILR,0.000384,RRSLR
7,RRILR,0.000279,RRILL
8,RRILR,0.000384,RLILR
9,RRILR,0.000524,RRIRR


In [97]:
aaseqs=[]
for i,row in df2.iterrows():
    seq1=CommonMotifProb.translateRNA(row.SourceSeq)
    seq2=CommonMotifProb.translateRNA(row.TargetSeq)
    aaseqs.append([seq1,row.ProbWeight,seq2])
    
dfaa=pd.DataFrame(aaseqs,columns=columns)

In [98]:
newaadictdata=[]
alreadychecked=[]
for i,row in dfaa.iterrows():
    if row.SourceSeq+row.TargetSeq not in alreadychecked and row.SourceSeq!=row.TargetSeq and '*' not in row.SourceSeq :

        dftemp=dfaa[(dfaa['SourceSeq']==row.SourceSeq)&(dfaa['TargetSeq']==row.TargetSeq)]
        probs=dftemp['ProbWeight']
        probssum=probs.sum()
        newaadictdata.append([row.SourceSeq, probssum, row.TargetSeq])
        alreadychecked.append(row.SourceSeq+row.TargetSeq)

In [99]:
dfnewaa=pd.DataFrame(newaadictdata,columns=columns)

In [100]:
dfnewaa.to_csv('SeqNetworkLandscapeAminoAcidIdenticalAASeq2.csv')

In [60]:
df2.to_csv('SeqNetworkLandscapeIdenticalAA.csv')

In [5]:
df['StartCodon'].value_counts()

AGGAGGATATTGAGG    194433
AGGAGGAGTTTGAGG    192800
CGACGAATCCTCCGA    147295
CGACGATCCCTCCGA    138217
Name: StartCodon, dtype: int64

In [54]:
df['StartCodon'].value_counts()

AGGACCAGTTTGAGG    178884
CGATGTTTCCTCCGA    160950
CGTCGTTTCAGCCCC    141745
CGTCGTAGCAGCCCC    136524
Name: StartCodon, dtype: int64

In [33]:
#Need to figure out a good way to represent this sequence space as a network and to compare the spaces of two different 
#mutation rates
print alltargetCodons

['AGGAGGGGTCGCAACGCTGCA' 'AGGGGGGGTCGCAACGGTGCA' 'AGGAGGGGTCGCAACGGCGCA'
 ..., 'AGGAGTGGTATCAAAGGTGCA' 'AGGAAGGGTATCAACGATGCA'
 'AGGATGGGAATCAACGGTGCA']


In [12]:
def uniqueAA(ntlist):
    AAlist=[]
    for i in ntlist:
        peptide=CommonMotifProb.translateRNA(i)
        if peptide not in AAlist:
            AAlist.append(peptide)
    return AAlist

In [14]:
#AGGAGGATATTGAGG
#CGACGAATCCTCCGA
df2=df[df['StartCodon']=='CGACGAATCCTCCGA']
df3=df[df['StartCodon']=='AGGAGGATATTGAGG']
print len(df2.TargetCodon.unique())
print len(df3.TargetCodon.unique())
print len(uniqueAA(df2.TargetCodon.unique()))
print len(uniqueAA(df3.TargetCodon.unique()))

607
870
260
426


In [108]:
#ilestart='AGGAGGATATTGAGG'
#argstart='CGACGAATCCTCCGA'

ilestart='CGACGAATCCTCCGA'
argstart='AGGAGGATATTGAGG'

fullDF=df[(df['StartCodon']==ilestart)]# | (df['StartCodon']==argstart)]

alltargetCodons=fullDF.TargetCodon.unique()



columns=['NodeSeq','ParentSeq','Count','Motif']


motif1=createmotif([['R'],['R'],['I'],['P'],['R']])
print motif1
datalist=[]



tempDF=fullDF


for TCodon in alltargetCodons:
    #print TCodon
    TcodonCount=0
    if TCodon!=ilestart and TCodon != argstart:
        protseq=CommonMotifProb.translateRNA(TCodon)
        ismotif=ismotifmatch(protseq,motif1)

        TcodonCount+=int(sum(tempDF[tempDF['TargetCodon']==TCodon]['Count']))
        #print TcodonCount
        if ismotif:
            motifValue=1
        else:
            motifValue=0

        if SeqMatch(ilestart,TCodon)==SeqMatch(argstart,TCodon):
            parentseq=3
        elif SeqMatch(ilestart,TCodon)<SeqMatch(argstart,TCodon):
            parentseq=1
        elif SeqMatch(ilestart,TCodon)>SeqMatch(argstart,TCodon):
            parentseq=2
        else:
            parentseq=0


        datalist.append([protseq,parentseq,TcodonCount,motifValue])

            
motifmutDF=pd.DataFrame(datalist,columns=columns)

[['R'], ['R'], ['I'], ['P'], ['R']]


In [106]:
motifmutDF

,NodeSeq,ParentSeq,Count,Motif
0,RRILR,2,12952,0
1,RRIVR,2,28143,0
2,RMILR,2,47950,0
3,RTILR,2,55901,0
4,RRTLR,2,223073,0
5,RRISR,2,259903,0
6,RRILM,2,26107,0
7,RSILR,2,37272,0
8,RRILR,2,251741,0
9,RRVLR,2,199551,0


In [80]:
motifmutDF
newmotifmutdata=[]
alreadychecked=[]
for t in [1,2]:
    alreadychecked=[]
    for i,row in motifmutDF.iterrows():
        if row.NodeSeq not in alreadychecked and row.NodeSeq != 'RRILR':
            dftemp=motifmutDF[(motifmutDF['NodeSeq']==row.NodeSeq)&(motifmutDF['ParentSeq']==t)]
            probs=dftemp['Count']
            probssum=probs.sum()
            newmotifmutdata.append([row.NodeSeq, row.ParentSeq, probssum,row.Motif])
            alreadychecked.append(row.NodeSeq)
            
newmotifmutDF=pd.DataFrame(newmotifmutdata,columns=columns)

In [103]:
motifmutDF
newmotifmutdata=[]
alreadychecked=[]
alreadychecked=[]
for i,row in motifmutDF.iterrows():
    if row.NodeSeq not in alreadychecked and row.NodeSeq != 'RRILR':
        dftemp=motifmutDF[(motifmutDF['NodeSeq']==row.NodeSeq)]
        probs=dftemp['Count']
        probssum=probs.sum()
        newmotifmutdata.append([row.NodeSeq, row.ParentSeq, probssum,row.Motif])
        alreadychecked.append(row.NodeSeq)
            
newmotifmutDF=pd.DataFrame(newmotifmutdata,columns=columns)

In [81]:
newmotifmutDF

,NodeSeq,ParentSeq,Count,Motif
0,RRVLR,1,194849,0
1,RRIRR,1,36930,0
2,RRILL,1,25836,0
3,R*ILR,1,17031,0
4,RRIIR,1,15924,0
5,RRIL*,1,29235,0
6,RRFLR,1,26341,0
7,RRLLR,1,12372,0
8,RQILR,1,27603,0
9,RRIPR,1,237489,0


In [107]:
motifs=motifmutDF[motifmutDF['Motif']==1]

one= motifs[motifs['ParentSeq']==1]
two= motifs[motifs['ParentSeq']==2]

print one['Count'].sum()
print two['Count'].sum()

0
387


In [109]:
motifs=motifmutDF[motifmutDF['Motif']==1]

one= motifs[motifs['ParentSeq']==1]
two= motifs[motifs['ParentSeq']==2]

print one['Count'].sum()
print two['Count'].sum()

237489
0


In [104]:
newmotifmutDF.to_csv('NetworkDataIdenticalAA_DetailsAminoAcidSeq2.csv')